In [1]:
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
import sys  
import random
import math 

# 預處理
讀取dataset以及進行簡單預處理。

In [2]:
data = pd.read_csv("./input/Iris.csv")                      
                                                                  
data = data.to_numpy()                                            
data = data[:,1:] #　remove column 'Id'                            
train_dataset, val_dataset = train_test_split(data, test_size=0.2)

# 建立decision tree的類別
我一共建立了Node與DecisionTree兩個類別。

## class Node

我將節點分為internal node（root node也被歸類於此）以及leaf node兩種。

### Internal node
Internal node用於將輸入資料分類（進行決策），假設輸入資料為`X`，我們可以依據輸入資料的特徵`X[feature_idx]`與是否小於等於閥值`threshold`，來決定輸入資料`X`要往左子節`left`或右子節點`right`前進。

### Leaf node
當有資料經過決策（internal nodes）後來到leaf node的話，則代表該筆資料被分類到第`class_type`個種類。

In [3]:
# class of node
class Node:
    def __init__(self, left=None, right=None, feature_idx=None, threshold=None, class_type=None):
        # Only be used at internal node
        self.left = left
        self.right = right
        self.feature_idx = feature_idx
        self.threshold = threshold
        
        # Only be used at leaf node
        self.class_type = class_type
        
        

## class DecisionTree

建立`DecisionTree`之初，我們會先定義幾個進行擬合時會用到的參數，這些參數會影響到樹是否會over/under fitting，以及影響擬合時間。參數如下：

* `max_depth`
    定義樹的最大深度
* `iter_feature_cnt`
    若選擇過大的`iter_feature_cnt`，代表每次增加節點時會考慮較多的特徵與可能的閥值，這可能導致擬合時間過長以及over fitting的發生。但若`iter_feature_cnt`過小則可能導致under fitting。
* `min_sample_cnt`
    進行擬合時，若節點的data數量小於`min_sample_cnt`則不繼續進行分割。

我也將Gini Impurity與entropy列於下方：
### Gini Impurity
$$\displaystyle GiniImpurity=1-\sum _{i=1}^{J}p_{i}^{2}$$

### Entropy
$$\displaystyle Enttopy=-\sum p(x)\log \left({p(x)}\right)$$

In [4]:
# class of DecisionTree
class DecisionTree:
    def __init__(self, max_depth, iter_feature_cnt, min_sample_cnt, information_type):
        self.root = None
        self.dice = None
        self.max_depth = max_depth
        # How many features will be choosed in each iteration of creating a new node
        # Larger iter_feature_cnt can get better precision. But it will need more time spending on fitting.
        self.iter_feature_cnt = iter_feature_cnt
        # If the sample count is less than min_sample_cnt, stop making decision.
        self.min_sample_cnt = min_sample_cnt
        self.information_type = information_type
        
    def fit(self, dataset):
        self.tree_init(dataset)
        self.root = self.create_node(dataset, 0)
        
    def tree_init(self, dataset):
        # The dice is used for randomly pick feature
        self.dice = range(dataset.shape[1] - 1)
    
    def create_node(self, dataset, curr_depth):
        # Use recursion to create the whole decision tree
        if curr_depth <= self.max_depth and len(dataset) >= self.min_sample_cnt and len(np.unique(dataset[:,-1])) > 1:
            ig, feature_idx, threshold, left_dataset, right_dataset = self.best_split(dataset)
            # Avoid dicision tree doesn't be splited
            # If information gain equals to 0, that means the dataset doesn't be splited
            if ig > 0: 
                # Create internal node
                return Node(left=self.create_node(left_dataset, curr_depth+1),
                            right=self.create_node(right_dataset, curr_depth+1),
                            feature_idx=feature_idx, 
                            threshold=threshold)

        # Current node is leaf node if current node doesn't need to be splited
        # Pick out the most frequent type in current dataset as the predicted type
        values, counts = np.unique(dataset[:,-1], return_counts=True)
        most_frequent_type = values[np.argmax(counts)]
        return Node(class_type=most_frequent_type)

    def roll_dice(self):
        # Used for randomly choose features
        self.dice = random.sample(self.dice, len(self.dice))
        
    def best_split(self, dataset):
        self.roll_dice()
        max_ig = -1 * sys.float_info.max
        max_feature_idx = None
        max_threshold = None
        
        # Pick out the feature and threshold with largest infromation gain as desicion 
        for i in range(self.iter_feature_cnt):
            curr_feature_idx = self.dice[i]
            for curr_threshold in np.unique(dataset[:,curr_feature_idx]):
                left_dataset, right_dataset = self.split(dataset, curr_feature_idx, curr_threshold)
                if len(left_dataset) > 0 and len(right_dataset) > 0:
                    curr_ig = self.compute_ig(dataset[:,-1], left_dataset[:,-1], right_dataset[:,-1])
                    if curr_ig > max_ig:
                        max_left_dataset, max_right_dataset = left_dataset, right_dataset
                        max_feature_idx = curr_feature_idx
                        max_threshold = curr_threshold
                        max_ig = curr_ig
        return max_ig, max_feature_idx, max_threshold, max_left_dataset, max_right_dataset
                    
                
    def split(self, dataset, feature_idx, threshold):
        # If the value of the feature is less and equal than threshold, splited to left_dataset. Otherwise, splited to right_dataset.
        left_dataset = np.array([row for row in dataset if row[feature_idx]<=threshold])
        right_dataset = np.array([row for row in dataset if row[feature_idx]>threshold])
        
        return left_dataset, right_dataset
    
    def compute_ig(self, y, left_y, right_y):
        weight_l = len(left_y) / len(y)
        weight_r = len(right_y) / len(y)
        if self.information_type == 'gini':
            return self.gini(y) - weight_l * self.gini(left_y) - weight_r * self.gini(right_y)
        elif self.information_type == 'entropy':
            return self.entropy(y) - weight_l * self.entropy(left_y) - weight_r * self.entropy(right_y)
        else:
            raise ValueError("Parameter 'information_type' for DecisionTree.compute_ig is invalid")
        
    def entropy(self, y):
        data_cnt = len(y)
        info = 0
        for cls in np.unique(y):
            p_x = (len(y[y == cls]) / data_cnt)
            info -= p_x * math.log(p_x)
        return info
            
    def gini(self, y):
        info = 1
        data_cnt = len(y)
        for cls in np.unique(y):
            info -= (len(y[y == cls]) / data_cnt) ** 2
        return info
    
    def predict_all(self, X):
        y = [self.predict_single(x) for x in X]
        return y
    
    def predict_single(self, x):
        curr_node = self.root

        while curr_node.class_type == None:
            if x[curr_node.feature_idx] <= curr_node.threshold:
                curr_node = curr_node.left
            else:
                curr_node = curr_node.right
        return curr_node.class_type
    
    def print_precision(self, y, pred_y):
        print(len([y[i] for i in range(len(y)) if y[i]==pred_y[i]]) / len(y))
    
    def print_tree(self):
        
        # Create binary tree
        binary_tree_list = [[self.root]]
        depth = 0
        while len(binary_tree_list) > depth:
            l = []
            for node in binary_tree_list[depth]:
                if node != None:
                    l.append(node.left)
                    l.append(node.right)
                else:
                    l.append(None)
                    l.append(None)
            # If there ara at least one node in current depth
            if len(l) > len([ele for ele in l if ele == None]):
                binary_tree_list.append(l)
            depth += 1
        
        # Print tree
        depth = 0
        for node_list in binary_tree_list:
            print('Depth: ' + str(depth))
            node_idx = 0
            for node in node_list:
                if node != None:
                    print('Node index: ' + str(node_idx) + '\t Feature index: ' + str(node.feature_idx), '\t Threshold: ' + str(node.threshold))
                node_idx += 1
            depth += 1
            print()
            

# 擬合與訓練

最後則是擬合與訓練的部分，各位可以自己調整以下幾個參數，來看對訓練成效的影響。我自己試了一下，要選取比較極端的數值（如`iter_feature_cnt=1`），才有可能讓precision比較低。

這裡我簡單的寫了一個print函式，讓各位可以簡單地看一下訓練出來的樹會是什麼樣子。
我將我們訓練出來的樹用complete binary tree的方式來print出來。假設深度=2且node index = 3的節點，其父節點為深度=2<b><font color=#FF0000>-1</font></b>=1且node index = 3<b><font color=#FF0000>//2</font></b>=1。也就是說，我們可以<b><font color=#FF0000>藉由將子節點的深度減1且node index除2來得到父節點的深度與node index</b>。

In [5]:
max_depth = 5
iter_feature_cnt = 4 # This number can't larger than 4. It will lead to error.
min_sample_cnt = 50
information_type = 'gini' # gini or entropy


tree = DecisionTree(max_depth, iter_feature_cnt, min_sample_cnt, information_type)
tree.fit(train_dataset)
pred_y = tree.predict_all(val_dataset)
tree.print_precision(val_dataset[:,-1], pred_y)
tree.print_tree()

0.8666666666666667
Depth: 0
Node index: 0	 Feature index: 2 	 Threshold: 1.9

Depth: 1
Node index: 0	 Feature index: None 	 Threshold: None
Node index: 1	 Feature index: 3 	 Threshold: 1.7

Depth: 2
Node index: 2	 Feature index: None 	 Threshold: None
Node index: 3	 Feature index: None 	 Threshold: None

